In [1]:
import torch

In [2]:
print(torch.cuda.is_available())

True


In [3]:
print(torch.cuda.device_count())

1


In [4]:
print(torch.cuda.get_device_name(0))

NVIDIA GeForce RTX 2050


In [6]:
if torch.cuda.is_available():
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached: ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Memory Usage:
Allocated: 0.0 GB
Cached:  0.0 GB


In [1]:
import pandas as pd

In [8]:
data = pd.read_csv(r"D:\Final_Year_Project\activity.csv")

In [9]:
data.head()

,1,2,3,4,5,6,7,8,9,10,...,1431,1432,1433,1434,1435,1436,1437,1438,1439,1440
0,4.065750,2.611625,2.788000,2.856250,5.141000,3.336875,0.420875,1.219375,0.523625,2.043250,...,7.423625,9.361625,5.565500,5.009750,3.337625,5.798875,2.302250,3.588750,4.083250,5.273625
1,0.166500,0.659250,1.977750,1.082625,0.079375,2.752375,1.546875,0.105750,0.058125,0.068000,...,0.158750,1.210750,0.441250,0.139375,0.107375,0.088250,0.062375,0.076125,0.086375,0.703500
2,2.440875,3.575750,4.724625,1.805500,4.467000,0.689000,2.895125,1.501000,2.101875,7.109125,...,6.157250,5.648000,7.784625,10.218125,7.646375,6.293125,8.520125,5.878125,5.795875,2.765750
3,5.095125,4.464250,6.074250,3.681625,4.690375,5.219750,6.147000,5.841625,9.058875,4.367875,...,10.470375,13.975000,13.481125,12.461000,13.888125,15.284500,14.235750,8.947500,6.828250,5.142750
4,1.230625,1.244375,0.852750,0.849125,0.109875,0.423125,1.051375,0.604625,0.069375,0.046125,...,2.975625,3.240000,4.677375,3.971875,3.626625,2.136500,0.925125,0.716500,1.564250,2.683875


In [11]:
data.mean(axis=1)

0        5.589662
1        5.110138
2        9.639063
3       10.905679
4        9.458740
          ...    
3027     5.190809
3028    12.144934
3029     8.887959
3030     7.366064
3031     7.201005
Length: 3032, dtype: float64

In [2]:
data2 = pd.read_csv(r"D:\Final_Year_Project\NHANES_FPCA_processed.csv")

In [3]:
data2.head()

,Age,Race,BMI,sex,Mobility,mortstat,diabetes.y,poverty_level,Asthma,Arthritis,...,act_mean,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9
0,80,3,33.9,0,2,1,0,4.28,0,1,...,5.589662,-2.491978,-0.868686,0.012114,-0.236829,0.698056,-0.189774,-0.032791,0.714461,0.118543
1,80,3,28.5,0,2,1,0,1.10,0,1,...,5.110138,-2.282571,1.434962,-0.180516,-1.044364,-0.345166,-0.099149,-0.134443,-0.902558,0.040988
2,62,1,26.0,1,2,0,0,1.38,0,1,...,9.639063,2.474805,-0.804625,0.021003,-1.030999,0.135622,-1.062733,-0.293907,-0.160872,-0.243265
3,65,3,26.7,1,2,0,1,0.00,0,1,...,10.905679,4.098556,-1.181728,-1.272489,0.803585,-0.566605,-0.602084,-1.074687,-0.589760,1.072204
4,77,1,30.6,1,1,1,1,0.97,0,1,...,9.458740,2.678961,1.652537,-0.174773,-0.402270,-0.323741,-0.480506,-0.298026,-0.349509,-0.270276


In [9]:
import pandas as pd
import numpy as np
from itertools import product
from sklearn.preprocessing import StandardScaler
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.metrics import concordance_index_censored
from sksurv.util import Surv

# Load split 1
df_train = pd.read_csv("D:/Final_Year_Project/splits_final/train_split_1.csv")
df_test  = pd.read_csv("D:/Final_Year_Project/splits_final/train_split_1.csv")

factor_cols = ["Race", "sex", "Mobility", "diabetes.y", "Asthma", "Arthritis", 
               "heart_failure", "coronary_heart_disease", "angina", "stroke", 
               "thyroid", "bronchitis", "cancer"]

# Convert categoricals
for col in factor_cols:
    df_train[col] = df_train[col].astype("category")
    df_test[col] = df_test[col].astype("category")

# Create survival labels
y_train = Surv.from_dataframe("mortstat", "time_mort", df_train)
y_test  = Surv.from_dataframe("mortstat", "time_mort", df_test)

# Drop unused columns
drop_cols = ["mortstat", "time_mort",'PC1', 'PC2', 'PC3', 'PC4',
       'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'dyn_feat_0', 'dyn_feat_1',
       'dyn_feat_2', 'dyn_feat_3', 'dyn_feat_4', 'dyn_feat_5', 'dyn_feat_6',
       'dyn_feat_7', 'dyn_feat_8', 'dyn_feat_9']
X_train = pd.get_dummies(df_train.drop(columns=drop_cols), drop_first=True)
X_test  = pd.get_dummies(df_test.drop(columns=drop_cols), drop_first=True)

In [10]:
X_train.columns

Index(['Age', 'BMI', 'poverty_level', 'act_mean', 'Race_2', 'Race_3', 'Race_4',
       'Race_6', 'Race_7', 'sex_1', 'Mobility_2', 'diabetes.y_1', 'Asthma_1',
       'Arthritis_1', 'heart_failure_1', 'coronary_heart_disease_1',
       'angina_1', 'stroke_1', 'thyroid_1', 'bronchitis_1', 'cancer_1'],
      dtype='object')

In [15]:
# Align columns between train/test
X_train, X_test = X_train.align(X_test, join="left", axis=1, fill_value=0)

# Standardize
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# Hyperparameter grid
param_grid = {
    "n_estimators": [500],
    "learning_rate": [0.1],
    "max_depth": [1],
    "subsample": [0.5]
}

# Grid search manually on test set C-index
results = []
for n, lr, d, ss in product(param_grid["n_estimators"],
                            param_grid["learning_rate"],
                            param_grid["max_depth"],
                            param_grid["subsample"]):

    model = GradientBoostingSurvivalAnalysis(
        n_estimators=n,
        learning_rate=lr,
        max_depth=d,
        subsample=ss,
        random_state=42
    )
    model.fit(X_train_scaled, y_train)
    risk_scores = -model.predict(X_test_scaled)
    c_index = concordance_index_censored(
        y_test["mortstat"], y_test["time_mort"], risk_scores
    )[0]
    results.append({
        "n_estimators": n,
        "learning_rate": lr,
        "max_depth": d,
        "subsample": ss,
        "c_index": c_index
    })

# Convert to DataFrame and sort
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="c_index", ascending=False)
print("Top configurations based on test-set C-index:")
print(results_df.head())

Top configurations based on test-set C-index:
   n_estimators  learning_rate  max_depth  subsample   c_index
0           500            0.1          1        0.5  0.203723


In [19]:
X_test_scaled

array([[ 1.46722963, -1.69226175, -0.00537262, ..., -0.53331788,
         2.99656868, -0.55051146],
       [ 0.82213275,  0.32270219, -1.26778624, ...,  1.87505431,
         2.99656868, -0.55051146],
       [-1.32819019, -1.52317386, -0.63045121, ...,  1.87505431,
        -0.33371503, -0.55051146],
       ...,
       [-1.1131579 , -0.73409708, -0.91847762, ...,  1.87505431,
        -0.33371503,  1.81649261],
       [ 0.60710045,  0.20997694, -1.26778624, ..., -0.53331788,
        -0.33371503, -0.55051146],
       [ 0.28455201, -0.38183065, -0.82042607, ..., -0.53331788,
        -0.33371503, -0.55051146]], shape=(2425, 21))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.util import Surv

# Load one split (e.g., split 1)
df_train = pd.read_csv("D:/Final_Year_Project/splits_final/train_split_1.csv")
df_test  = pd.read_csv("D:/Final_Year_Project/splits_final/test_split_1.csv")

factor_cols = ["Race", "sex", "Mobility", "diabetes.y", "Asthma", "Arthritis", 
               "heart_failure", "coronary_heart_disease", "angina", "stroke", 
               "thyroid", "bronchitis", "cancer"]

# Convert to categorical
for col in factor_cols:
    df_train[col] = df_train[col].astype("category")
    df_test[col] = df_test[col].astype("category")

# Construct structured array for survival
y_train = Surv.from_dataframe("mortstat", "time_mort", df_train)

# Prepare features
drop_cols = ["mortstat", "time_mort",]
X_train = pd.get_dummies(df_train.drop(columns=drop_cols), drop_first=True)

# Set up pipeline
pipe = make_pipeline(
    StandardScaler(),
    GradientBoostingSurvivalAnalysis(random_state=42)
)

# Define hyperparameter grid
param_grid = {
    "gradientboostingsurvivalanalysis__n_estimators": [100, 200, 300],
    "gradientboostingsurvivalanalysis__learning_rate": [0.01, 0.05, 0.1],
    "gradientboostingsurvivalanalysis__max_depth": [2, 3, 4],
    "gradientboostingsurvivalanalysis__subsample": [0.7, 1.0],
}

# Define grid search with 5-fold CV
gcv = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=5,
    scoring="neg_brier_score",  # or use custom scorer with concordance_index_censored
    n_jobs=-1
)

# Fit grid search
gcv.fit(X_train, y_train)

# Best parameters
print("Best Parameters:\n", gcv.best_params_)


In [27]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.preprocessing import OneHotEncoder
from sksurv.metrics import concordance_index_censored
from sksurv.util import Surv
from sklearn.model_selection import train_test_split



In [33]:
# --------------------------
# LOAD YOUR DATA (split 1)
# --------------------------
df = pd.read_csv("D:/Final_Year_Project/NHANES_FPCA_processed.csv")

# --------------------------
# SETUP: survival object & features
# --------------------------
# Categorical variables
factor_cols = ["Race", "sex", "Mobility", "diabetes.y", "Asthma", "Arthritis",
               "heart_failure", "coronary_heart_disease", "angina", "stroke",
               "thyroid", "bronchitis", "cancer"]
for col in factor_cols:
    df[col] = df[col].astype("category")

# Define survival outcome
y_structured = Surv.from_dataframe("mortstat", "time_mort", df)

# Drop non-feature columns
drop_cols = ["time_mort", "mortstat", 'PC1', 'PC2', 'PC3', 'PC4',
       'PC5', 'PC6', 'PC7', 'PC8', 'PC9']

X = df.drop(columns=drop_cols)

# One-hot encode categorical variables
encoder = OneHotEncoder()
X_encoded = encoder.fit_transform(X)

# --------------------------
# TRAIN-TEST SPLIT
# --------------------------
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_structured,
                                                    test_size=0.2, random_state=42)


est_cph_tree = GradientBoostingSurvivalAnalysis(n_estimators=1000, learning_rate=0.05, max_depth=1, random_state=42)
est_cph_tree.fit(X_train, y_train)
cindex = est_cph_tree.score(X_test, y_test)
print(cindex)

0.7875827145919414 3


In [35]:
import os
import pandas as pd
import numpy as np
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.preprocessing import OneHotEncoder
from sksurv.util import Surv

# ========= CONFIG =========
data_path = "D:/Final_Year_Project/splits_final"  # change if needed
train_prefix = "train_split_"
test_prefix = "test_split_"
n_splits = 100

factor_cols = ["Race", "sex", "Mobility", "diabetes.y", "Asthma", "Arthritis",
               "heart_failure", "coronary_heart_disease", "angina", "stroke",
               "thyroid", "bronchitis", "cancer"]

drop_cols = ["mortstat", "time_mort",'PC1', 'PC2', 'PC3', 'PC4',
       'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'dyn_feat_0', 'dyn_feat_1',
       'dyn_feat_2', 'dyn_feat_3', 'dyn_feat_4', 'dyn_feat_5', 'dyn_feat_6',
       'dyn_feat_7', 'dyn_feat_8', 'dyn_feat_9']

# ========= RESULT STORAGE =========
c_index_list = []

# ========= LOOP THROUGH SPLITS =========
for i in range(1, n_splits + 1):
    train_file = os.path.join(data_path, f"{train_prefix}{i}.csv")
    test_file = os.path.join(data_path, f"{test_prefix}{i}.csv")

    try:
        # Load data
        df_train = pd.read_csv(train_file)
        df_test = pd.read_csv(test_file)

        # Categoricals to category dtype
        for col in factor_cols:
            df_train[col] = df_train[col].astype("category")
            df_test[col] = df_test[col].astype("category")

        # Survival outcome
        y_train = Surv.from_dataframe("mortstat", "time_mort", df_train)
        y_test = Surv.from_dataframe("mortstat", "time_mort", df_test)

        # Drop unnecessary columns
        X_train = df_train.drop(columns=drop_cols, errors="ignore")
        X_test = df_test.drop(columns=drop_cols, errors="ignore")

        # One-hot encode
        encoder = OneHotEncoder()
        X_train_enc = encoder.fit_transform(X_train)
        X_test_enc = encoder.transform(X_test)

        # Model
        model = GradientBoostingSurvivalAnalysis(
            n_estimators=1000,
            learning_rate=0.05,
            max_depth=1,
            random_state=42
        )

        model.fit(X_train_enc, y_train)
        c_index = model.score(X_test_enc, y_test)
        c_index_list.append(c_index)
        print(f"Split {i}: C-index = {c_index:.4f}")

    except Exception as e:
        print(f"Error on split {i}: {e}")
        c_index_list.append(np.nan)

# ========= SAVE RESULTS =========
df_result = pd.DataFrame({
    "split": list(range(1, n_splits + 1)),
    "c_index": c_index_list
})
# df_result.to_csv("gbsm_cindex_from_100_splits.csv", index=False)

print(f"\n✅ Done! Average C-index: {np.nanmean(c_index_list):.4f}")

Split 1: C-index = 0.7845
Split 2: C-index = 0.8067
Split 3: C-index = 0.7944
Split 4: C-index = 0.7661
Split 5: C-index = 0.7479
Split 6: C-index = 0.7744
Split 7: C-index = 0.7522
Split 8: C-index = 0.7671
Split 9: C-index = 0.7463
Split 10: C-index = 0.7842
Split 11: C-index = 0.7738
Split 12: C-index = 0.8046
Split 13: C-index = 0.7746
Split 14: C-index = 0.7842
Split 15: C-index = 0.7851
Split 16: C-index = 0.7429
Split 17: C-index = 0.7635
Split 18: C-index = 0.7754
Split 19: C-index = 0.7646
Split 20: C-index = 0.8004
Split 21: C-index = 0.7774
Split 22: C-index = 0.7622
Split 23: C-index = 0.7845
Split 24: C-index = 0.7908
Split 25: C-index = 0.7693
Split 26: C-index = 0.7961
Split 27: C-index = 0.7647
Split 28: C-index = 0.7697
Split 29: C-index = 0.7998
Split 30: C-index = 0.7372
Split 31: C-index = 0.8019
Split 32: C-index = 0.8007
Split 33: C-index = 0.7579
Split 34: C-index = 0.7973
Split 35: C-index = 0.7893
Split 36: C-index = 0.7689
Split 37: C-index = 0.7751
Split 38: 

In [36]:
df_result.to_csv("gbsm_cindex_from_100_splits.csv", index=False)